In [2]:
%pip install azure-ai-generative[evaluate,index,promptflow]==1.0.0b3

  Using cached azure_ai_generative-1.0.0b3-py3-none-any.whl.metadata (9.9 kB)
  Using cached azureml_metrics-0.0.54-py3-none-any.whl.metadata (13 kB)
  Using cached azureml_dataprep-5.1.6-py3-none-any.whl.metadata (2.2 kB)
  Using cached azureml_fsspec-1.3.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached azureml_mlflow-1.56.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached mmh3-4.1.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached Markdown-3.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached tika-2.6.0-py3-none-any.whl
  Using cached pypdf-3.17.4-py3-none-any.whl.metadata (7.5 kB)
  Using cached unstructured-0.11.8-py3-none-any.whl.metadata (26 kB)
  Using cached azure_search_documents-11.4.0b11-py3-none-any.whl.metadata (22 kB)
  Using cached promptflow_vectordb-0.2.10-py3-none-any.whl.metadata (3.8 kB)
  Using cached azureml_dataprep_native-41

  error: subprocess-exited-with-error
  
  × Building wheel for faiss-cpu (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      running bdist_wheel
      running build
      running build_py
      running build_ext
      building 'faiss._swigfaiss' extension
      swigging faiss\faiss\python\swigfaiss.i to faiss\faiss\python\swigfaiss_wrap.cpp
      swig.exe -python -c++ -Doverride= -I/usr/local/include -Ifaiss -doxygen -DSWIGWIN -module swigfaiss -o faiss\faiss\python\swigfaiss_wrap.cpp faiss\faiss\python\swigfaiss.i
      error: command 'swig.exe' failed: None
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for faiss-cpu
ERROR: Could not build wheels for faiss-cpu, which is required to install pyproject.toml-based projects


In [3]:
import os
from azure.ai.resources.client import AIClient
from azure.ai.resources.operations._index_data_source import (
    LocalSource,
    ACSOutputConfig,
)
from azure.ai.generative.index import build_index
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv()

contoso_search = os.environ["SEARCH_SERVICE"]
index_name = "contoso-manuals-index"

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
openai_deployment = "text-embedding-ada-002"

path_to_data = "./manuals"

ModuleNotFoundError: No module named 'azure.ai.generative'

In [ ]:
# Set up environment variables for cog search SDK
os.environ["AZURE_AI_SEARCH_ENDPOINT"] = contoso_search

client = AIClient.from_config(credential = DefaultAzureCredential())

# Use the same index name when registering the index in AI Studio
index = build_index(
    output_index_name=index_name,
    vector_store="azure_cognitive_search",
    embeddings_model=f"azure_open_ai://deployment/{openai_deployment}/model/{openai_deployment}",
    data_source_url="/products",
    index_input_config=LocalSource(input_data=path_to_data),
    acs_config=ACSOutputConfig(
        acs_index_name=index_name,
    ),
)

In [ ]:
# register the index so that it shows up in the project
cloud_index = client.indexes.create_or_update(index)

print(f"Created index '{cloud_index.name}'")
print(f"Local Path: {index.path}")
print(f"Cloud Path: {cloud_index.path}")

In [ ]:
%pip uninstall azure-ai-generative[evaluate,index,promptflow]==1.0.0b3 -y

In [ ]:
%pip install azure-search-documents==11.4.0